In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
import os
import tensorflow as tf
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [3]:
df=pd.read_csv("/content/drive/MyDrive/bully_blocker/data.csv")

In [4]:
df.head()

,comments,tagging
0,Get fucking real dude.,1
1,She is as dirty as they come and that crook ...,1
2,why did you fuck it up. I could do it all day...,1
3,Dude they dont finish enclosing the fucking s...,1
4,WTF are you talking about Men? No men thats n...,1


In [5]:
df.head().style.set_properties(**{'background-color':'Aqua',
                                     'color': 'purple'})

,comments,tagging
0,Get fucking real dude.,1
1,She is as dirty as they come and that crook Rengel the Dems are so fucking corrupt it's a joke. Make Republicans look like ...,1
2,why did you fuck it up. I could do it all day too. Let's do it when you have an hour. Ping me later to sched writing a book here.,1
3,Dude they dont finish enclosing the fucking showers. I hate half assed jobs. Whats the reasononing behind it? Makes no sense.,1
4,WTF are you talking about Men? No men thats not a menage that's just gay.,1


In [6]:
df.shape

(20001, 2)

In [7]:
df['tagging'].value_counts()

,count
tagging,
0,12179
1,7822


In [8]:
label_counts = {}
for label in df['tagging']:
    if label in label_counts:
        label_counts[label] += 1
    else:
        label_counts[label] = 1

In [9]:
label_counts

{1: 7822, 0: 12179}

In [10]:
def get_sequences(texts, tokenizer, train=True, max_seq_length=None):
    sequences = tokenizer.texts_to_sequences(texts)

    if train == True:
        max_seq_length = np.max(list(map(lambda x: len(x), sequences)))

    sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_seq_length, padding='post')

    return sequences


In [11]:
def preprocess_inputs(df):
    df = df.copy()


    y = df['tagging']
    X = df['comments']

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)

    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=30000)

    tokenizer.fit_on_texts(X_train)

    X_train = get_sequences(X_train, tokenizer, train=True)
    X_test = get_sequences(X_test, tokenizer, train=False, max_seq_length=X_train.shape[1])

    return X_train, X_test, y_train, y_test

In [12]:
X_train, X_test, y_train, y_test = preprocess_inputs(df)

In [13]:
X_train

array([[   1,  537,  579, ...,    0,    0,    0],
       [ 116, 1574,  149, ...,    0,    0,    0],
       [ 164, 5268,  892, ...,    0,    0,    0],
       ...,
       [  18,   37,   42, ...,    0,    0,    0],
       [  38,  241,    1, ...,    0,    0,    0],
       [  28, 4690,    0, ...,    0,    0,    0]], dtype=int32)

In [14]:
y_train.value_counts()

,count
tagging,
0,8554
1,5446


In [15]:
X_train.shape

(14000, 207)

In [16]:
inputs = tf.keras.Input(shape=(207,))

embedding = tf.keras.layers.Embedding(
    input_dim=30000,
    output_dim=64
)(inputs)

flatten = tf.keras.layers.Flatten()(embedding)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(flatten)


model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)


model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 207)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 207, 64)             │       1,920,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 13248)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │          13,249 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,933,249 (7.37 MB)

 Trainable params: 1,933,249 (7.37 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
start = time.time()
history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)
end = time.time()
print(end-start)

Epoch 1/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.6370 - auc: 0.5976 - loss: 0.6455 - val_accuracy: 0.7071 - val_auc: 0.8108 - val_loss: 0.5407
Epoch 2/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8344 - auc: 0.9147 - loss: 0.4143 - val_accuracy: 0.8032 - val_auc: 0.8773 - val_loss: 0.4383
Epoch 3/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9356 - auc: 0.9846 - loss: 0.2126 - val_accuracy: 0.8364 - val_auc: 0.9053 - val_loss: 0.3844
Epoch 4/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9705 - auc: 0.9958 - loss: 0.1222 - val_accuracy: 0.8464 - val_auc: 0.9133 - val_loss: 0.3713
Epoch 5/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9841 - auc: 0.9985 - loss: 0.0753 - val_accuracy: 0.8482 - val_auc: 0.9177 - val_loss: 0.3768
Epoch 6/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9909 - auc: 0.9993 - loss: 0.0493 - val_accuracy: 0.8546 - val_auc: 0.9182 - val_loss: 0.3851
Epoch 7/10
350/350 ━━━━━━━━━━━━━━━━━━━━

In [18]:
y_pred = model.predict(X_test)
y_pred_labels = y_pred.argmax(axis=1)
y_true = y_test
y_pred_labels = (y_pred > 0.5).astype(int)  # For binary classification, adjust if multi-class

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix:\n", cm)

# Precision
precision = precision_score(y_true, y_pred_labels)
print("Precision:", precision)

# Recall
recall = recall_score(y_true, y_pred_labels)
print("Recall:", recall)

# F1 Score
f1 = f1_score(y_true, y_pred_labels)
print("F1 Score:", f1)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Confusion Matrix:
 [[3219  406]
 [ 527 1849]]
Precision: 0.8199556541019956
Recall: 0.7781986531986532
F1 Score: 0.7985316346361477


In [19]:
inputs = tf.keras.Input(shape=(207,))

embedding = tf.keras.layers.Embedding(
    input_dim=30000,
    output_dim=128
)(inputs)

flatten = tf.keras.layers.Flatten()(embedding)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(flatten)


model2 = tf.keras.Model(inputs=inputs, outputs=outputs)


model2.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)


model2.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 207)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_1 (Embedding)              │ (None, 207, 128)            │       3,840,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 26496)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │          26,497 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,866,497 (14.75 MB)

 Trainable params: 3,866,497 (14.75 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
start = time.time()
history = model2.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

end = time.time()
print(end-start)

Epoch 1/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.6341 - auc: 0.6131 - loss: 0.6407 - val_accuracy: 0.7275 - val_auc: 0.8187 - val_loss: 0.5309
Epoch 2/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8589 - auc: 0.9296 - loss: 0.3669 - val_accuracy: 0.8114 - val_auc: 0.8933 - val_loss: 0.4072
Epoch 3/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9583 - auc: 0.9929 - loss: 0.1527 - val_accuracy: 0.8364 - val_auc: 0.9132 - val_loss: 0.3699
Epoch 4/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9804 - auc: 0.9984 - loss: 0.0788 - val_accuracy: 0.8475 - val_auc: 0.9178 - val_loss: 0.3694
Epoch 5/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9885 - auc: 0.9993 - loss: 0.0526 - val_accuracy: 0.8621 - val_auc: 0.9203 - val_loss: 0.3778
Epoch 6/100
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9916 - auc: 0.9996 - loss: 0.0363 - val_accuracy: 0.8625 - val_auc: 0.9191 - val_loss: 0.4020
Epoch 7/100
350/350 ━━━━━━━━━━━━━━

In [21]:
y_pred = model2.predict(X_test)
y_pred_labels = y_pred.argmax(axis=1)
y_true = y_test
y_pred_labels = (y_pred > 0.5).astype(int)  # For binary classification, adjust if multi-class

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix:\n", cm)

# Precision
precision = precision_score(y_true, y_pred_labels)
print("Precision:", precision)

# Recall
recall = recall_score(y_true, y_pred_labels)
print("Recall:", recall)

# F1 Score
f1 = f1_score(y_true, y_pred_labels)
print("F1 Score:", f1)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Confusion Matrix:
 [[3149  476]
 [ 445 1931]]
Precision: 0.8022434565849605
Recall: 0.8127104377104377
F1 Score: 0.8074430273886682


In [22]:
inputs = tf.keras.Input(shape=(207,))

embedding = tf.keras.layers.Embedding(
    input_dim=30000,
    output_dim=128
)(inputs)

flatten = tf.keras.layers.Flatten()(embedding)

outputs1 = tf.keras.layers.Dense(16, activation='tanh')(flatten)
# outputs2 = tf.keras.layers.Dense(32, activation='tanh')(outputs1)
outputs3 = tf.keras.layers.Dense(64, activation='tanh')(outputs1)
outputs4 = tf.keras.layers.Dense(32, activation='tanh')(outputs3)
outputs5 = tf.keras.layers.Dense(8, activation='tanh')(outputs4)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(outputs5)



model3 = tf.keras.Model(inputs=inputs, outputs=outputs)


model3.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)


model3.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 207)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 207, 128)            │       3,840,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 26496)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │         423,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           1,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 8)                   │             264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,267,393 (16.28 MB)

 Trainable params: 4,267,393 (16.28 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
start = time.time()
history = model3.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=12,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)
end = time.time()
print(end-start)

Epoch 1/12
350/350 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6160 - auc: 0.5017 - loss: 0.6684 - val_accuracy: 0.5932 - val_auc: 0.5000 - val_loss: 0.6758
Epoch 2/12
350/350 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6204 - auc: 0.5038 - loss: 0.6643 - val_accuracy: 0.5932 - val_auc: 0.5000 - val_loss: 0.6760
Epoch 3/12
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6104 - auc: 0.4921 - loss: 0.6688 - val_accuracy: 0.5932 - val_auc: 0.5000 - val_loss: 0.6761
Epoch 4/12
350/350 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6160 - auc: 0.4904 - loss: 0.6664 - val_accuracy: 0.5932 - val_auc: 0.5000 - val_loss: 0.6765
16.263482332229614


In [24]:
y_pred = model.predict(X_test)
y_pred_labels = y_pred.argmax(axis=1)
y_true = y_test
y_pred_labels = (y_pred > 0.5).astype(int)


cm = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix:\n", cm)


precision = precision_score(y_true, y_pred_labels)
print("Precision:", precision)


recall = recall_score(y_true, y_pred_labels)
print("Recall:", recall)


f1 = f1_score(y_true, y_pred_labels)
print("F1 Score:", f1)

188/188 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Confusion Matrix:
 [[3219  406]
 [ 527 1849]]
Precision: 0.8199556541019956
Recall: 0.7781986531986532
F1 Score: 0.7985316346361477


In [48]:
import tensorflow as tf

class FCLayer(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, use_bias=True, dropout_rate=0.0, batch_norm=False, order="LABDN", **kwargs):
        super(FCLayer, self).__init__(**kwargs)
        self.units = units
        self.activation = activation
        self.use_bias = use_bias
        self.dropout_rate = dropout_rate
        self.batch_norm = batch_norm
        self.order = order

        self.dense = tf.keras.layers.Dense(self.units, use_bias=self.use_bias)
        if self.activation:
            self.activation_fn = tf.keras.layers.Activation(self.activation)
        else:
            self.activation_fn = None

        if self.batch_norm:
            self.batch_norm_layer = tf.keras.layers.BatchNormalization()

        if self.dropout_rate > 0:
            self.dropout = tf.keras.layers.Dropout(self.dropout_rate)

        self.normalization_layer = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))

    def call(self, inputs, training=False):
        x = inputs

        for layer_code in self.order:
            if layer_code == 'L':
                x = self.dense(x)
            elif layer_code == 'A' and self.activation_fn:
                x = self.activation_fn(x)
            elif layer_code == 'B' and self.batch_norm:
                x = self.batch_norm_layer(x, training=training)
            elif layer_code == 'D' and self.dropout_rate > 0:
                x = self.dropout(x, training=training)
            elif layer_code == 'N':
                x = self.normalization_layer(x)

        return x

def build_backbone():
    inputs = tf.keras.Input(shape=(207,))
    embedding = tf.keras.layers.Embedding(input_dim=30000, output_dim=128)(inputs)
    flatten = tf.keras.layers.Flatten()(embedding)
    return inputs, flatten

class SigmoidHead(tf.keras.layers.Layer):
    def __init__(self, units):
        super(SigmoidHead, self).__init__()
        self.dense = tf.keras.layers.Dense(units)
        self.sigmoid = tf.keras.layers.Activation('sigmoid')

    def call(self, inputs):
        x = self.dense(inputs)
        return self.sigmoid(x)

def build_model():
    inputs, backbone_output = build_backbone()

    adapter = FCLayer(units=256, activation='relu', batch_norm=True, dropout_rate=0.3, order="LABDN")(backbone_output)

    outputs = SigmoidHead(units=1)(adapter)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

model4 = build_model()

model4.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

model4.summary()


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)          │ (None, 207)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_12 (Embedding)             │ (None, 207, 128)            │       3,840,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_12 (Flatten)                 │ (None, 26496)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc_layer_14 (FCLayer)                │ (None, 256)                 │       6,784,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sigmoid_head_9 (SigmoidHead)         │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,624,513 (40.53 MB)

 Trainable params: 10,624,001 (40.53 MB)

 Non-trainable params: 512 (2.00 KB)

In [49]:
start = time.time()
history = model4.fit(
    X_train, y_train,
    epochs=3,
    batch_size=64,
    validation_data=(X_test, y_test)
)
end = time.time()
print(end-start)

Epoch 1/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.4971 - auc: 0.5011 - loss: 0.7004 - val_accuracy: 0.6036 - val_auc: 0.5167 - val_loss: 0.6710
Epoch 2/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9955 - auc: 0.9997 - loss: 0.3207 - val_accuracy: 0.6032 - val_auc: 0.5159 - val_loss: 0.8855
Epoch 3/3
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 0.0527 - val_accuracy: 0.6032 - val_auc: 0.5163 - val_loss: 0.8072
13.997915506362915


In [46]:
y_pred = model4.predict(X_test)
y_pred_labels = y_pred.argmax(axis=1)
y_true = y_test
y_pred_labels = (y_pred > 0.5).astype(int)

cm = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix:\n", cm)

from sklearn.metrics import accuracy_score
val_accuracy_sklearn = accuracy_score(y_test, y_pred_labels)
print(f'Scikit-learn Validation Accuracy: {val_accuracy_sklearn:.4f}')

precision = precision_score(y_true, y_pred_labels)
print("Precision:", precision)

recall = recall_score(y_true, y_pred_labels)
print("Recall:", recall)

f1 = f1_score(y_true, y_pred_labels)
print("F1 Score:", f1)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Confusion Matrix:
 [[   3 3622]
 [   0 2376]]
Scikit-learn Validation Accuracy: 0.3964
Precision: 0.39613204401467156
Recall: 1.0
F1 Score: 0.5674707427752568


In [53]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd


data = pd.read_csv("/content/drive/MyDrive/bully_blocker/data.csv")

max_words = 10000
max_len = 100

X = data['comments'].values
y = data['tagging'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=max_len)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_words, 128, input_length=max_len),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',
                       tf.keras.metrics.Precision(),
                       tf.keras.metrics.Recall()])

history = model.fit(X_train_padded, y_train, epochs=8, batch_size=32, validation_split=0.2)


test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test_padded, y_test)


test_f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1 Score: {test_f1}")


y_pred_prob = model.predict(X_test_padded)
y_pred = (y_pred_prob > 0.5).astype("int32")


print(classification_report(y_test, y_pred))


Epoch 1/8


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


400/400 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.6505 - loss: 0.6096 - precision: 0.5722 - recall: 0.2584 - val_accuracy: 0.7644 - val_loss: 0.4655 - val_precision: 0.6531 - val_recall: 0.8242
Epoch 2/8
400/400 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8485 - loss: 0.3430 - precision: 0.7985 - recall: 0.8193 - val_accuracy: 0.8284 - val_loss: 0.4025 - val_precision: 0.8047 - val_recall: 0.7307
Epoch 3/8
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9316 - loss: 0.1846 - precision: 0.9150 - recall: 0.9142 - val_accuracy: 0.8547 - val_loss: 0.4044 - val_precision: 0.7912 - val_recall: 0.8446
Epoch 4/8
400/400 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9604 - loss: 0.1124 - precision: 0.9467 - recall: 0.9531 - val_accuracy: 0.8553 - val_loss: 0.4718 - val_precision: 0.7845 - val_recall: 0.8592
Epoch 5/8
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9746 - loss: 0.0784 - precision: 0.9670 - recall: 0.9676 - val_accuracy: 0.8744 - val_loss: 0.4943 - val_p

In [58]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/bully_blocker/data.csv")

max_words = 10000
max_len = 100

X = data['comments'].values
y = data['tagging'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=max_len)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len)

def evaluate_model(model, X_test, y_test):
    test_loss, test_acc, test_precision, test_recall = model.evaluate(X_test, y_test)
    test_f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall)

    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_acc}")
    print(f"Test Precision: {test_precision}")
    print(f"Test Recall: {test_recall}")
    print(f"Test F1 Score: {test_f1}")

    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > 0.5).astype("int32")
    print(classification_report(y_test, y_pred))

# Model 1: RNN
def build_model_rnn():
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(max_words, 128, input_length=max_len),
        tf.keras.layers.SimpleRNN(64),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',
                                                                       tf.keras.metrics.Precision(),
                                                                       tf.keras.metrics.Recall()])
    return model

# Model 2: LSTM
def build_model_lstm():
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(max_words, 128, input_length=max_len),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',
                                                                       tf.keras.metrics.Precision(),
                                                                       tf.keras.metrics.Recall()])
    return model

# Model 3: Encoder-Decoder
def build_model_encoder_decoder():
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(max_words, 128, input_length=max_len),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',
                                                                       tf.keras.metrics.Precision(),
                                                                       tf.keras.metrics.Recall()])
    return model

# Model 4: Transformer
def build_model_transformer():
    inputs = tf.keras.Input(shape=(max_len,))
    x = tf.keras.layers.Embedding(max_words, 128)(inputs)
    x = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=128)(x, x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',
                                                                       tf.keras.metrics.Precision(),
                                                                       tf.keras.metrics.Recall()])
    return model

# Training and Evaluating Each Model
for i, build_model in enumerate([build_model_rnn, build_model_lstm, build_model_encoder_decoder, build_model_transformer]):
    print(f"\nTraining Model {i+1}")
    print(build_model)
    model = build_model()
    st=time.time()
    model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_split=0.2)
    end=time.time()
    print(build_model)
    print("time taken" , end-st)
    evaluate_model(model, X_test_padded, y_test)



Training Model 1
<function build_model_rnn at 0x7c3be5fe7eb0>


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.6688 - loss: 0.6033 - precision_6: 0.6103 - recall_6: 0.3329 - val_accuracy: 0.8091 - val_loss: 0.4255 - val_precision_6: 0.7983 - val_recall_6: 0.6729
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.9309 - loss: 0.1883 - precision_6: 0.9248 - recall_6: 0.9004 - val_accuracy: 0.8622 - val_loss: 0.3729 - val_precision_6: 0.8167 - val_recall_6: 0.8267
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9804 - loss: 0.0638 - precision_6: 0.9792 - recall_6: 0.9711 - val_accuracy: 0.8566 - val_loss: 0.4586 - val_precision_6: 0.7835 - val_recall_6: 0.8657
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9846 - loss: 0.0513 - precision_6: 0.9808 - recall_6: 0.9796 - val_accuracy: 0.8413 - val_loss: 0.5320 - val_precision_6: 0.7548 - val_recall_6: 0.8690
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.9903 - loss: 0.0321 - precision_6: 0.9878 - recall_6: 0.

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.6545 - loss: 0.6017 - precision_7: 0.5929 - recall_7: 0.2949 - val_accuracy: 0.7669 - val_loss: 0.4685 - val_precision_7: 0.6851 - val_recall_7: 0.7274
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8588 - loss: 0.3295 - precision_7: 0.8149 - recall_7: 0.8253 - val_accuracy: 0.8100 - val_loss: 0.4250 - val_precision_7: 0.7616 - val_recall_7: 0.7356
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9278 - loss: 0.1961 - precision_7: 0.9094 - recall_7: 0.9093 - val_accuracy: 0.8381 - val_loss: 0.4321 - val_precision_7: 0.7582 - val_recall_7: 0.8495
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.9570 - loss: 0.1204 - precision_7: 0.9447 - recall_7: 0.9455 - val_accuracy: 0.8547 - val_loss: 0.4487 - val_precision_7: 0.7975 - val_recall_7: 0.8332
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9699 - loss: 0.0872 - precision_7: 0.9582 - recall_7: 0.9640 - val_accu

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


400/400 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.6524 - loss: 0.6112 - precision_8: 0.5983 - recall_8: 0.2706 - val_accuracy: 0.7759 - val_loss: 0.4633 - val_precision_8: 0.6667 - val_recall_8: 0.8332
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.8588 - loss: 0.3278 - precision_8: 0.8160 - recall_8: 0.8298 - val_accuracy: 0.8222 - val_loss: 0.4010 - val_precision_8: 0.7741 - val_recall_8: 0.7583
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9360 - loss: 0.1697 - precision_8: 0.9185 - recall_8: 0.9190 - val_accuracy: 0.8269 - val_loss: 0.4276 - val_precision_8: 0.7355 - val_recall_8: 0.8576
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.9669 - loss: 0.1006 - precision_8: 0.9553 - recall_8: 0.9595 - val_accuracy: 0.8606 - val_loss: 0.4845 - val_precision_8: 0.7946 - val_recall_8: 0.8592
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.9785 - loss: 0.0631 - precision_8: 0.9711 - recall_8: 0.9748 - val_

In [25]:
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))


dhtml('Thanks to Dr. Bam Bahadur Sinha sir, all code by Abhishek.')